<a href="https://colab.research.google.com/github/AvanishSiva/silicon-feed/blob/master/backend/py/SiliconFeedScrpr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 29.5 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,283 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,625 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import json
from google.colab import drive
from datetime import datetime
import time


In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
options = webdriver.ChromeOptions()

In [25]:
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [38]:
def scrpFromTechCrnch():
    # Output file path
    output_file_path = '/content/drive/MyDrive/siliconFeedScrpe.json'

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)
    print("Opening TechCrunch homepage...")
    driver.get('https://techcrunch.com/')

    articles = driver.find_elements(By.CSS_SELECTOR, '.loop-card__title-link')
    links = [article.get_attribute('href') for article in articles]
    print(f"Found {len(links)} articles to scrape.")

    all_articles_data = []

    for i, link in enumerate(links):
        print(f"Scraping article {i + 1}/{len(links)}: {link}")
        try:
            driver.get(link)
            time.sleep(2)

            title = driver.find_element(By.CLASS_NAME, 'article-hero__title').text
            category = driver.find_element(By.CLASS_NAME, 'is-taxonomy-category').text
            summary = driver.find_element(By.ID, 'speakable-summary').text

            contents_array = [
                content.text for content in driver.find_elements(By.CLASS_NAME, 'wp-block-paragraph')
            ]

            tags_div = driver.find_element(By.CLASS_NAME, 'tc23-post-relevant-terms__terms')
            tags = [tag.text for tag in tags_div.find_elements(By.TAG_NAME, 'a')]

            article_data = {
                'title': title,
                'category': category,
                'summary': summary,
                'contents': contents_array,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'url': link,
                'tags': tags,
                "source": "TechCrunch"
            }
            all_articles_data.append(article_data)

            print(f"Successfully scraped article {i + 1}: {title}")
        except Exception as e:
            print(f"Failed to scrape article {i + 1}: {e}")

    driver.quit()
    print("Finished scraping all articles.")

    try:
        with open(output_file_path, 'r') as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    existing_data.extend(all_articles_data)

    with open(output_file_path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

    print(f"Scraped data appended to {output_file_path}. Total articles saved: {len(existing_data)}")

In [39]:
scrpFromTechCrnch()

Opening TechCrunch homepage...
Found 82 articles to scrape.
Scraping article 1/82: https://techcrunch.com/2025/01/24/elevenlabs-has-raised-a-new-round-at-3b-valuation-led-by-iconiq-growth-sources-say/
Successfully scraped article 1: ElevenLabs has raised a new round at $3B+ valuation led by ICONIQ Growth, sources say
Scraping article 2/82: https://techcrunch.com/2025/01/24/the-ai-industrys-pace-has-researchers-stressed/
Successfully scraped article 2: The AI industry’s pace has researchers stressed
Finished scraping all articles.
Scraped data appended to /content/drive/MyDrive/siliconFeedScrpe.json. Total articles saved: 2
